# Cell Counting Template
For nuclei counting on Operetta/Omero
## Step1 check segmentation on a few example image.
Choose a few representative images on Omero and copy Image IDs into List.
This will open a window displaying the original image, segmentation and bounding
box in Napari. You can save images of the different layers as JPEGS.
 ## Step2 Run program across the selected Plates
 Choose plate IDs and add to Plate ID list
 This will generate two CSV files for combined counts per image/well and single cell data.

In [12]:
#Define Variables
# 1) Name of Datafolder -> Experiment
Project_Name='220420_XM_CellCount01'
# 2) Location where to save Data -> PD
PD='/Users/hh65/Desktop'
# 3) Sample Image ID -> ID List
ID_List=[295146]
# 3) Define concentrations used in experiment
Concentrations=[0,50,100,200,400,800,0,50,100,200,400,800]
# 4) Define Plate IDs for analysis
plate_list = [662]


In [1]:

%run ../Functions-Module.ipynb
import os
from omero.gateway import BlitzGateway
from tqdm import tqdm
import pandas as pd

# Make folder to save output data in chosen directory
# D_PATH = os.path.join(PD, Project_Name)
# os.makedirs(D_PATH, exist_ok=True)



Found model '2D_versatile_fluo' for 'StarDist2D'.
Metal device set to: Apple M1 Pro


2022-05-10 10:03:27.078194: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-10 10:03:27.078497: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [2]:
# Connect to Omero and get Plate name
conn = BlitzGateway('helfrid', 'Omero_21', host='ome2.hpc.susx.ac.uk')
conn.connect()


True

In [10]:
for ID in ID_List:
    object= conn.getObject("Image", ID)
    pixels=object.getPrimaryPixels()
    imgDAPI=pixels.getPlane(0,0,0)
    segmented_cells, cell_number=Stardist_Segmentation(imgDAPI)
    Napari_Display(imgDAPI, segmented_cells, cell_number)


2022-04-20 17:15:21.454450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
numeric.py (2446): elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [15]:
df=pd.DataFrame()
for plate_id in plate_list:
    dict1={'condition':[],'row':[],'column':[],'image':[],'cell_number':[]}
    plate = conn.getObject("Plate", plate_id)
    plate_name= plate.getName()
    wells = list(plate.listChildren())
    print (plate_name, well.id)
    df_sc=pd.DataFrame()
    for number, well in tqdm(enumerate(wells)):
        for number, image in enumerate(well.listChildren()):
            dict1['condition'].append(plate_name)
            dict1['row'].append(well.row)
            dict1['column'].append(well.column)
            dict1['image'].append(image.id)
            image = well.getImage(number)
            pixels=image.getPrimaryPixels()
            img=pixels.getPlane()
            df, cell_number = Stardist_Counting(img)
            dict1['cell_number'].append(cell_number)
            df['row']=well.row
            df['column']=well.column
            df_sc=pd.concat([df_sc,df])
    df_sc.to_csv(os.path.join(D_PATH,plate_name+'_scdata.csv'))
    df_cc = pd.DataFrame.from_dict(dict1)
    df_cc=df_cc.sort_values(['row','column'])

    df_cc.to_csv(os.path.join(D_PATH,plate_name+'_ccdata.csv'))


Index.idx.xml 4739


0it [00:00, ?it/s]2022-04-20 17:32:05.817607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
48it [10:05, 12.62s/it]
